# "머신러닝을 통한 접근 가이드" 목차
## 1. Library
## 2. Data Loading
## 3. Feature Engineering
### 3-1. Feature Generation
#### CODE SHARE WITH "파베르"님 
#### (https://dacon.io/competitions/official/235745/codeshare/2851?page=1&dtype=recent)
### 3-2. Feature Engineering
#### 3-2-1. Encoding
#### 3-2-2. Scailing
## 4. Modeling with Pycaret
## 5. Modeling with CatBoostRegressor

## 1. Library

In [1]:
# for "2. Data Loading"
import pandas as pd

# for "3-1. Feature Generation"
import numpy as np

# for "3-2. Feature Engineering"
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import RobustScaler, StandardScaler

# for "4. Modeling with Pycaret"
from pycaret.regression import *

# for "5. Modeling with CatBoostRegressor"
from catboost import CatBoostRegressor
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, StratifiedKFold

## 2. Data Loading

In [2]:
train = pd.read_csv('data/train0725(2).csv')
test = pd.read_csv('data/test0725(2).csv')
train = train.set_index("code")
test = test.set_index("code")

In [3]:
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

In [4]:
scailing_features = ['tot_pop',
                     'region',
                     'num_park',
                     'sub',
                     'bus',
                     'num_empty',
                     'M_fee_deposit',
                     'M_fee_rent',
                    ]

scaler = MinMaxScaler()
train.loc[:, scailing_features] = scaler.fit_transform(train[scailing_features])
test.loc[:, scailing_features] = scaler.transform(test[scailing_features])

## 3. Feature Engineering

### 3-1. Feature Generation
#### CODE SHARE WITH "파베르"님 
#### (https://dacon.io/competitions/official/235745/codeshare/2851?page=1&dtype=recent)
##### Feature Generation과 관련된 EDA는 위의 링크를 참조해주세요 :) 

# Modeling with Gradient Boosting Regressor

- Setting Data For Model

In [5]:
X = train.drop(columns = ['target'])
y = train['target']

- Hyper params Tuning

In [6]:
def objective(trial: Trial) -> float:
    params_cat = {
        "random_state": 42,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "verbose" : 1,
        "objective" : "MAE",
        "max_depth": trial.suggest_int("max_depth", 1, 16),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.8, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2)

    model = CatBoostRegressor(**params_cat)
    model.fit(
        X_tr,
        y_tr,
        eval_set=[(X_tr, y_tr), (X_val, y_val)],
        early_stopping_rounds=10,
        verbose=False,
    )

    cat_pred = model.predict(X_val)
    log_score = mean_absolute_error(y_val, cat_pred)
    
    return log_score

In [7]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="cat_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2021-07-29 09:17:04,693] A new study created in memory with name: cat_opt
[I 2021-07-29 09:17:05,148] Trial 0 finished with value: 127.12419611621829 and parameters: {'max_depth': 6, 'colsample_bylevel': 0.9901428612819833, 'subsample': 0.8123957592679836, 'min_child_samples': 62, 'max_bin': 246}. Best is trial 0 with value: 127.12419611621829.
[I 2021-07-29 09:17:05,360] Trial 1 finished with value: 111.54339434320154 and parameters: {'max_depth': 3, 'colsample_bylevel': 0.8116167224336399, 'subsample': 0.9063233020424546, 'min_child_samples': 62, 'max_bin': 413}. Best is trial 1 with value: 111.54339434320154.
[I 2021-07-29 09:17:05,644] Trial 2 finished with value: 126.01470052750791 and parameters: {'max_depth': 1, 'colsample_bylevel': 0.9939819704323989, 'subsample': 0.8827098485602951, 'min_child_samples': 25, 'max_bin': 254}. Best is trial 1 with value: 111.54339434320154.
[I 2021-07-29 09:17:05,850] Trial 3 finished with value: 128.66926777877237 and parameters: {'max_depth'

Best Score: 111.54339434320154
Best trial: {'max_depth': 3, 'colsample_bylevel': 0.8116167224336399, 'subsample': 0.9063233020424546, 'min_child_samples': 62, 'max_bin': 413}


In [8]:
cat_p = {'max_depth': 3, 'colsample_bylevel': 0.8116167224336399, 'subsample': 0.9063233020424546, 'min_child_samples': 62, 'max_bin': 413}
cat = CatBoostRegressor(**cat_p)

- StratifiedK-Fold for Regression

In [9]:
y_cat = pd.cut(y, 10, labels=range(10))
skf = StratifiedKFold(5)

preds = []
for tr_id, val_id in skf.split(X, y_cat) : 
    X_tr = X.iloc[tr_id]
    y_tr = y.iloc[tr_id]
    
    cat.fit(X_tr, y_tr, verbose = 0)
    
    pred = cat.predict(test)
    preds.append(pred)
cat_pred = np.mean(preds, axis = 0)

In [10]:
sample = pd.read_csv('data/sample_submission.csv')
sample['num'] = cat_pred
sample.to_csv('sub/cat0729_2.csv', index=False)

In [11]:
sample

,code,num
0,C1072,747.147877
1,C1128,1253.572347
2,C1456,497.986124
3,C1840,498.015540
4,C1332,1140.290280
5,C1563,1821.835199
6,C1794,868.200875
7,C1640,447.243854
8,C1377,332.426090
9,C2072,259.933127


In [12]:
sample

,code,num
0,C1072,747.147877
1,C1128,1253.572347
2,C1456,497.986124
3,C1840,498.015540
4,C1332,1140.290280
5,C1563,1821.835199
6,C1794,868.200875
7,C1640,447.243854
8,C1377,332.426090
9,C2072,259.933127
